## 一、数据处理讲解  
本部分代码在 ```preprocess.py```


### 1、数据结构

（1）原始数据：  
thchs30数据由wav文件和trn文件两大类组成，其中wav文件为实际语音数据，trn文件为标注数据。  
数据样例：  
"好容易 盼 来一 位 望望 这一片 荒芜 又 扭 身 而去"  
"hao3 rong2 yi4 pan4 lai2 yi1 wei4 wang4 wang5 zhe4 yi2 pian4 huang1 wu2 you4 niu3 shen1 er2 qu4"  
"h ao3 r ong2 ii i4 p an4 l ai2 ii i1 uu ui4 uu uang4 uu uang5 zh e4 ii i2 p ian4 h uang1 uu u2 ii iu4 n iu3 sh en1 ee er2 q v4"  
可见第一行为文本标注，第二行为音节标注，第三行为音素标注。  

（2）处理后的数据：  
处理后的数据按照说话人保存为npz文件，例如，第2号说话人的数据保存为data_2.npz。  
每一条数据包含：语音特征、文本标注、音节标注、音素标注和说话人id。

#### 数据结构示例

In [5]:
import numpy as np

file_in = './data_raw/data_thchs30/D11_892.wav.trn'
with open(file_in, 'r', encoding='utf8')as f:
    file_contents = f.read()
    print(file_contents)

npz_file = './data/data_2.npz'
npz_data = np.load(npz_file, allow_pickle=True)
print(npz_data['X'][0].shape)
print(npz_data['sen'][0])
print(npz_data['syllable'][0])
print(npz_data['phone'][0])
print(npz_data['speaker'][0])

好容易 盼 来一 位 望望 这一片 荒芜 又 扭 身 而去
hao3 rong2 yi4 pan4 lai2 yi1 wei4 wang4 wang5 zhe4 yi2 pian4 huang1 wu2 you4 niu3 shen1 er2 qu4
h ao3 r ong2 ii i4 p an4 l ai2 ii i1 uu ui4 uu uang4 uu uang5 zh e4 ii i2 p ian4 h uang1 uu u2 ii iu4 n iu3 sh en1 ee er2 q v4

(1600, 200, 1)
刘寿春崛出肮脏的尖须忘记把吞烟的手收下来用呆钝的眼睛望着他
[578, 915, 159, 466, 142, 12, 1184, 208, 432, 1100, 1045, 426, 21, 1022, 1121, 208, 914, 912, 1071, 1313, 1154, 193, 247, 208, 1123, 1300, 1045, 1220, 961]
[196, 147, 203, 103, 89, 125, 164, 186, 89, 78, 17, 208, 17, 140, 88, 164, 48, 206, 175, 205, 118, 164, 46, 27, 4, 204, 125, 207, 48, 140, 88, 203, 102, 203, 100, 206, 144, 196, 11, 207, 173, 140, 7, 140, 128, 140, 88, 207, 50, 164, 72, 205, 118, 209, 88, 204, 2]
2


### 2、读取及处理数据
读取及处理数据的逻辑如下：  
（1）遍历所有语料库  
（2）处理thchs30语料库  
（3）针对wav文件获取语音特征（统一转换为（1600，200，1）的数组）  
（4）针对trn文件获取标注数据  
（5）将数据添加至列表供后续存储

#### 读取数据函数示例

In [2]:
from util.utils import *
file_in = './data_raw/data_thchs30/D11_892.wav'
x_data = get_feature(file_in)
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1)
print(x_data.shape)

(1600, 200, 1)


### 3、存储数据
将读取的数据列表统一存储至npz文件中。
函数：```save_npz()```  
  
参数：```X_list:语音特征列表 ```  
&emsp;&emsp;&emsp;```sen_list:文本标注列表 ```  
&emsp;&emsp;&emsp;```syllable_list:音节标注列表 ```  
&emsp;&emsp;&emsp;```phone_list:音素标注列表 ```  
&emsp;&emsp;&emsp;```speaker_list:说话人id列表 ```  
&emsp;&emsp;&emsp;```speaker:要保存的说话人id ```  
  
返回：```无```  

#### 存储数据函数示例

In [2]:
from util.utils import *
file_in = './data_raw/data_thchs30/D11_892.wav'
x_data = get_feature(file_in)
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1)
X_list = [x_data]
sen_list = ['测试呀']
syllable_list = [['578, 915, 159, 466, 142, 12, 961']]
phone_list = [[196, 147, 203, 103, 89, 125, 164, 186, 89, 78, 17,2]]
speaker_list = ['test']
speaker = 'test'
print("Saving speaker data:", speaker)
save_npz(X_list, sen_list, syllable_list, phone_list, speaker_list, speaker)

Saving speaker data: test


## 二、数据处理执行  

In [ ]:
import numpy as np
import re, os, codecs, sys
from config import Config as cfg
from util.utils import *


#开始数据预处理
print("Begin to preprocess data!")
syllable_txt_list = get_syllable_txt_list(os.path.join(os.getcwd(), 'misc', 'syllable2char_cn.txt'))
syllable2phone_dict, phone2nbr_dict, nbr2phone_dict = get_phone_dict(os.path.join(os.getcwd(), 'misc', 'syllable2phone_cn.txt'))
print("syllable and phone length: ",len(syllable_txt_list), len(phone2nbr_dict))


#准备变量列表
names = locals()
for speaker in cfg.speaker_select_thchs30:
    names['X_list_%s' % speaker], names['syllable_list_%s' % speaker], names['phone_list_%s' % speaker], names['sen_list_%s' % speaker], names['speaker_list_%s' % speaker] = [], [], [], [], []


In [ ]:
#开始处理每个语料库
print("Begin to load corpus!")
for corpus in cfg.corpus:
    print("Processing corpus:", corpus)
    directory = os.path.join(cfg.rawdata_dir, corpus)
    for dirpath, _, filenames in os.walk(directory):
        for f in filenames:
            filename = os.path.join(dirpath, f)
            # print(filename, corpus)
            #判断文件名以wav结尾
            if ('wav' in filename[-3:] or 'WAV' in filename[-3:]):
                #处理thchs30语料库
                if(corpus == 'data_thchs30'):
                    #thchs30的trn格式：
                    #企业 依靠 技术 挖潜
                    #qi3 ye4 yi1 kao4 ji4 shu4 wa1 qian2
                    #q i3 ii ie4 ii i1 k ao4 j i4 sh u4 uu ua1 q ian2
                    file_label = filename + '.trn'
                    if(not os.path.exists(file_label)):continue

                    #特定录音人的选取
                    speaker = get_speaker(filename)
                    if(speaker not in cfg.speaker_select_thchs30):continue
                    
                    #获取语音特征
                    x_data = get_feature(filename)
                    x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1)
                    # print(x_data.shape)

                    #获取标注数据
                    with open(file_label, encoding='utf-8') as f:
                        lines = f.readlines()
                        sen = lines[0].strip('\r\n').strip('\n').replace(" ","")
                        syllable_l = lines[1].strip('\r\n').strip('\n').split(' ')
                        phone_l = []
                        for syllable in syllable_l:
                            phone_l.extend(syllable2phone_dict[syllable])
                        syllable_nbr_l = ref_list_to_nbr(syllable_l, syllable_txt_list)
                        phone_nbr_l = ref_dict_to_nbr(phone_l, phone2nbr_dict)
                
                #将数据添加至列表
                names['X_list_%s' % speaker].append(x_data)
                names['sen_list_%s' % speaker].append(sen)
                names['syllable_list_%s' % speaker].append(syllable_nbr_l)
                names['phone_list_%s' % speaker].append(phone_nbr_l)
                names['speaker_list_%s' % speaker].append(speaker)
    
    #每个speaker保存为一个npz文件
    for speaker in cfg.speaker_select_thchs30:
        print("Saving speaker data:", speaker)
        save_npz(names['X_list_%s' % speaker], names['sen_list_%s' % speaker], names['syllable_list_%s' % speaker], 
            names['phone_list_%s' % speaker], names['speaker_list_%s' % speaker], speaker)

print("Preprocess data done!")